# Import and definitions

In [1]:
# Jupyter Notebook 
from IPython.display import clear_output

# Additional utils
import numpy as np
import pprint
import uuid
import zmq
import matplotlib.pyplot as plt

# c3 Imports
import numpy as np
import tensorflow as tf
import c3po.hamiltonians as hamiltonians
from c3po.utils import log_setup
from c3po.simulator import Simulator as Sim
from c3po.optimizer import Optimizer as Opt
from c3po.experiment import Experiment as Exp

# Specify Initial Signal/Pulse and Bounds

### Initialize and set parameters + bounds of Components of the Signal/Pulse

In [ ]:
t_final = 50e-9

flattop_params = {
    'amp' : 15e6 * 2 * np.pi,
    'T_up' : 5e-9,
    'T_down' : 45e-9,
    'xy_angle' : 0,
    'freq_offset' : 0e6 * 2 * np.pi
}

flattop_bounds = {
    'amp': [1e3*2*np.pi, 1e9*2*np.pi],
    'T_up': [2e-9, 145e-9],
    'T_down': [2e-9, 145e-9],
    'xy_angle': [-np.pi, np.pi],
    'freq_offset': [-1e9*2*np.pi, 1e9*2*np.pi]
}

flattop2_params = {
    'amp' : 15e6 * 2 * np.pi,
    'T_up' : 50e-9,
    'T_down' : 75e-9,
    'xy_angle' : np.pi/2,
    'freq_offset' : 0e6 * 2 * np.pi
}

carrier_parameters = {
    'freq' : 6e9 * 2 * np.pi
}


def my_flattop(t, params):
    t_up = params['T_up']
    t_down = params['T_down']
    return flattop(t, t_up, t_down)


# Component 1 of the signal
flat = Comp(
    desc = "pulse1",
    shape = my_flattop,
    params = flattop_params,
    bounds = flattop_bounds
)

# Component 2 of the signal. Another flattop componen which uses the same bounds as the first one
flat2 = Comp(
    desc = "pulse2",
    shape = my_flattop,
    params = flattop2_params,
    bounds = flattop_bounds
)

# Component 3 of the signal; the carrier doesn't have a shape only parameters
carr = Comp(
    desc = "carrier",
    params = carrier_parameters
)

### Build Signal with Components 

In [ ]:
qubit_X = control.Instruction(
    name="Q1X",
    t_start=0.0,
    t_end=t_final,
    channels=["d1"]
)
qubit_X.add_component(gauss_env, "d1")
qubit_X.add_component(carr, "d1")

gates = control.GateSet()
gates.add_instruction(qubit_X)

Display the list of registered components and their uuids.

In [ ]:
sig = IQ()
sig.t_start = 0
sig.t_end = 150e-9 # Should be compatible with bounds
sig.res = [1e9, 1e12]

sig.calc_slice_num()
sig.create_ts()

sig.comps = comps

sig.save_params_to_history("initial")

# Calibration

In [ ]:
optim = Optimizer()

In [ ]:
opt_map = {
    'amp' : [(sig.get_uuid(), flat.get_uuid())],
    'T_up' : [(sig.get_uuid(), flat.get_uuid())],
    'T_down' :[(sig.get_uuid(), flat.get_uuid())],
    'xy_angle': [(sig.get_uuid(), flat.get_uuid())],
    'freq_offset': [(sig.get_uuid(), flat.get_uuid())]
}

In [ ]:
opt_map_full = {}
pprint.pprint(sig.generate_opt_map(opt_map_full))

In [ ]:
opt_params = optim.get_corresponding_signal_parameters([sig], opt_map)

In [ ]:
pprint.pprint(opt_params)

In [ ]:
initial_spread = [ 5e6*2*np.pi, 5e6*2*np.pi, 5e-9, 5e-9, 5e-9, 5e-9, 0.1, 0.1, 20e6*2*np.pi]

opt_settings = {
    'CMA_stds': initial_spread,
    'ftarget' : 1e-4,
    'popsize' : 5
}

In [ ]:
def evaluate_signals_dbg(samples_rescaled):
    """
    Evaluate function to debug an optimizer call. Returns random results for all input samples.
    """
    return np.random.rand(len(samples_rescaled))

### User functions

In [ ]:
######################
# Display functions  #
######################
def plot_IQ(ts, Is, Qs):
    plt.cla()
    plt.plot(ts / 1e-9, Is)
    plt.plot(ts / 1e-9, Qs)
    plt.legend(("I", "Q"))
    plt.ylabel('I/Q')
    plt.xlabel('Time[ns]')
    plt.tick_params('both',direction='in')
    plt.tick_params('both', direction='in')
    fig.canvas.draw()
    fig.canvas.flush_events()

##########################
# Labview communication  #
##########################
def LV_send_pulse(search_id, pulse_id, pulse):
    request = {
            'search_id': search_id,
            'pulse_id': pulse_id,
            'pulse': pulse,
            'fidelity': 0,
            'do_stop': False
            }
    socketreq.send_json(request)
    

def LV_recieve_fidelity(search_id):
    receive_message = socketreq.recv_string()        
    reply = socketrep.recv_json()
    socketrep.send_string(search_id)    
    return reply['fidelity']

#####################
# Evaluate function #
#####################

def evaluate_signals(samples_rescaled):
    """
    Evaluate function
    """
    global search_id
    global fig
    
    infidelities = []
    for sigs in samples_rescaled:
        pulse_id = str(uuid.uuid4())
        sig = sigs[0]
        IQ = sig.get_IQ()
        ts = sig.ts[0]
        
        Is = IQ['I']
        Qs = IQ['Q']
        
        sig.plot_IQ(ts, Is, Qs)
        fig.canvas.draw()
        fig.canvas.flush_events()
        
        non0_mask = np.nonzero(Is * Qs)
        pulse = {}
        pulse['I'] = list(Is[non0_mask])  
        pulse['Q'] = list(Qs[non0_mask])
        pulse['omegas'] = [IQ['omega']]
        pulse['carrier_amp'] = IQ['amp']
        LV_send_pulse(search_id, pulse_id, pulse)
        fidelity = LV_recieve_fidelity(search_id)
        infidelities.append(1-fidelity)
    return infidelities

In [ ]:
optim = Optimizer()

In [ ]:
##########################
# Labview communication  #
##########################

calibration_daemon_experiment_URI = "tcp://127.0.0.1:5559"
calibration_daemon_searcher_URI = "tcp://127.0.0.1:5560"

   
rcvtimeout = 30000
search_id = str(uuid.uuid4())
print(f"I am the calibration searcher.\nMy ID for this run is {search_id}\n")

# Start communication
print(f"Connecting to client {calibration_daemon_experiment_URI} ... ", flush=True, end='')
context = zmq.Context()
socketrep = context.socket(zmq.REP)

socketrep.bind(calibration_daemon_searcher_URI)

socketreq = context.socket(zmq.REQ)
socketreq.setsockopt(zmq.LINGER, 0)  #NEW Added by Edwar to flush the queue
socketreq.connect(calibration_daemon_experiment_URI)

socketrep.RCVTIMEO = rcvtimeout  # added timeout to kill rcv if nothing comes
socketreq.RCVTIMEO = rcvtimeout
print(f"done\n\n", flush=True)

In [ ]:
%matplotlib
plt.rcParams['figure.dpi'] = 100
plt.ion()
plt.show()
global fig
fig = plt.figure(figsize=(5, 4))


optim.optimize_signal(
    signals = [sig],
    opt_map = opt_map,
    opt = 'cmaes',
    settings = opt_settings,
    calib_name = 'test',
    eval_func = evaluate_signals
    )

In [ ]:
socketrep.unbind(calibration_daemon_searcher_URI)
socketreq.disconnect(calibration_daemon_experiment_URI)